# Assignment 3

In [1]:
# let's start by bringing in pandas and numpy
import pandas as pd
import numpy as np
# Filter all warnings.
import warnings
warnings.filterwarnings('ignore')

## Question 1
Load the energy data from the file [Datasets/EnergyIndicators.xls](Datasets/EnergyIndicators.xls), which is a list of indicators of [energy supply and renewable electricity production](Datasets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Also, we make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

`"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"`

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

In [2]:
# so let's bring in the data via pandas
Energy = pd.read_excel('Datasets/EnergyIndicators.xls', header=None)
# we don't want the header and footer, so we skip them
Energy = Energy[18:245].reset_index().rename(columns={2: 'Country', 3: 'Energy Supply', 4: 'Energy Supply per Capita', 5: '% Renewable'})
# and we drop useless column
Energy.drop({'index', 0, 1}, inplace=True, axis=1)

# now we replace '...' as NaN value
Energy['Energy Supply'] = np.where(Energy['Energy Supply'] == '...', np.nan, Energy['Energy Supply'])
Energy['Energy Supply per Capita'] = np.where(Energy['Energy Supply per Capita'] == '...', np.nan, Energy['Energy Supply per Capita'])
# let's transforme the Energy Supply units by Giga Joules
Energy['Energy Supply'] = Energy['Energy Supply'] * 10**6

# there're names in country column that have numbers or other undisarible characters
Energy['Country'] = Energy['Country'].replace('\d*', '', regex=True)
Energy['Country'] = Energy['Country'].replace('\s*\(.*\)', '', regex=True)

# let's change the name of some countries
old_names = ["Republic of Korea", "United States of America",
             "United Kingdom of Great Britain and Northern Ireland", 
             "China, Hong Kong Special Administrative Region"]
new_names = ["South Korea", "United States", "United Kingdom", "Hong Kong"]
Energy['Country'] = Energy['Country'].replace(old_names,new_names)
# let's switch Energy as a DataFrame
Energy = pd.DataFrame(Energy)
Energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321000000,10,78.66928
1,Albania,102000000,35,100
2,Algeria,1959000000,51,0.55101
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9000000,121,88.69565


Next, load the GDP data from the file [Datasets/world_bank.csv](Datasets/world_bank.csv), which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

In [3]:
GDP = pd.read_csv('Datasets/world_bank.csv', skiprows=4)
# let's change the names of some countries
old_names = ["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"]
new_names = ["South Korea", "Iran", "Hong Kong"]
GDP['Country Name'] = GDP['Country Name'].replace(old_names, new_names)

GDP = GDP.rename(columns={'Country Name': 'Country'})
GDP = pd.DataFrame(GDP)
# now, we are interested in the last 10 years data, namely, from 2006 to 2015
GDP.drop(GDP.iloc[:, 1:50], inplace=True, axis=1)
GDP.head()

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file [Datasets/scimagojr-3.xlsx](Datasets/scimagojr-3.xlsx), which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [4]:
# First, let's call the data
ScimEn = pd.read_excel('Datasets/scimagojr-3.xlsx')
ScimEn = pd.DataFrame(ScimEn)
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [5]:
# let's merge these three DataFrame on Country column
submerge = pd.merge(ScimEn, Energy, how='right', on='Country')
Merge = pd.merge(submerge, GDP, how='right', on='Country')
# and we take just the first 15th in the rank
Merge = Merge[Merge['Rank'] <= 15]
# let's set country column as index
Merge = Merge.set_index('Country')
Merge

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14.0,8831.0,8725.0,90765.0,15606.0,10.28,107.0,5386000000,231,11.81081,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15.0,8668.0,8596.0,60702.0,14396.0,7.00,86.0,12149000000,59,69.64803,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6.0,17899.0,17620.0,215003.0,40930.0,12.01,149.0,10431000000,296,61.94543,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,127191000000,93,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
Germany,7.0,17027.0,16831.0,140566.0,27426.0,8.26,126.0,13261000000,165,17.90153,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
Spain,12.0,9428.0,9330.0,123336.0,23964.0,13.08,115.0,4923000000,106,37.96859,1.414823e+12,1.468146e+12,1.484530e+12,1.431475e+12,1.431673e+12,1.417355e+12,1.380216e+12,1.357139e+12,1.375605e+12,1.419821e+12
France,9.0,13153.0,12973.0,130632.0,28601.0,9.93,114.0,10597000000,166,17.02028,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7920000000,124,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
India,8.0,15005.0,14841.0,128763.0,37209.0,8.58,115.0,33195000000,26,14.96908,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12


## Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [6]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [7]:
# so we start by making the union between dataframes
subunion = pd.merge(ScimEn, Energy, how='outer', left_on='Country', right_on='Country')
Union = pd.merge(subunion, GDP, how='outer', left_on='Country', right_on='Country')
Union = Union.set_index('Country')

# now we make the intersection between them
subinter = pd.merge(ScimEn, Energy, how='inner', left_on='Country', right_on='Country')
Inter = pd.merge(subinter, GDP, how='inner', left_on='Country', right_on='Country')
Inter = Inter.set_index('Country')

# so, the amount of lost entries is
lost_entries = int(len(Union.index) - len(Inter.index))
lost_entries

158

## Question 3
What are the top 15 countries for average GDP over the last 10 years?

In [8]:
# Merge has all the information we need, so we just take the mean value of each rows, i.e., of each country
avgGDP = Merge[['2006','2007','2008','2009','2010',
                '2011','2012','2013','2014','2015']].mean(axis=1).sort_values(ascending=False)
avgGDP

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

## Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

In [9]:
# country with the 6th largest average GDP
country_sixth = avgGDP.index[avgGDP==avgGDP[5]][0]
country_sixth

'United Kingdom'

In [10]:
# 2006 GDP value
V2006 = Merge.loc[country_sixth][10]
# 2015 GDP value
V2015 = Merge.loc[country_sixth][19]
GDP_over_10 = float(abs(V2015 - V2006))
GDP_over_10

246702696075.3999

## Question 5
What is the mean energy supply per capita?

In [11]:
# We only want the Energy Supply per Capita column ans its average
Mean_Energy_Supply_per_Capita = Merge['Energy Supply per Capita'].mean()
Mean_Energy_Supply_per_Capita

157.6

## Question 6
What country has the maximum % Renewable and what is the percentage?

In [12]:
# let's bring in the '% Renewable' column and its maximun
Renewable = Merge[['% Renewable']]
Max_Renewable = Merge['% Renewable'].max()
# and the name of the country which belongs the maximun
Name_max_rene = Renewable.index[Renewable['% Renewable']==Renewable['% Renewable'].max()][0]
(Name_max_rene, Max_Renewable)

('Brazil', 69.64803)

## Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

In [13]:
# let's bring in the relevant columns
Citations = Merge[['Citations','Self-citations']]
# we add the column 'Citations Ratio'
Citations['Citations Ratio'] = Citations['Self-citations']/Citations['Citations']
# this is the maximun ratio
Max_Citation = Citations['Citations Ratio'].max()
# and the respective country
Name_max_cita = Citations['Citations Ratio'].index[Citations['Citations Ratio']==Max_Citation][0]
result = (Name_max_cita, float(Max_Citation))
result

('China', 0.6893126179389422)

## Question 8
Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

In [14]:
# we use only the following columns
Energies = Merge[['Energy Supply', 'Energy Supply per Capita']]
# now we estimate the population
Energies['Population'] = Energies['Energy Supply']/Energies['Energy Supply per Capita']
# now we sort data on descending way,
Name_Ener = Energies['Population'].sort_values(ascending=False).reset_index()
# and sought the third most populous country
Name_Ener = Name_Ener['Country'][2]
Name_Ener

'United States'

## Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

In [15]:
# let's bring in scipy to deal with correlations
import scipy.stats as stats

In [16]:
# We already know the countries' population, so let's compute the citable documents per capita
Documents = Merge[['Energy Supply per Capita', 'Citable documents']]
Documents['Population'] = Energies['Population']
Documents['Citable Documents per Capita'] = Documents['Citable documents']/Documents['Population']
# so the regresion of these column is
corr, pval = stats.pearsonr(Documents["Citable Documents per Capita"],Documents["Energy Supply per Capita"])
(corr, pval)

(0.7940010435442942, 0.0004083648953039728)

## Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

In [17]:
# first let's define a function that makes that
def renewable(x, y):
    if (x < y):
        return 0
    return 1
# so, the % Renewable median is
Median_Renew = Renewable['% Renewable'].median()
Median_Renew

17.02028

In [18]:
# applying the function
Renewable['HighRenew'] = Merge['% Renewable'].apply(lambda x: renewable(x, Median_Renew))
# Renewable = Renewable['HighRenew']
Renewable

,% Renewable,HighRenew
Country,,
Australia,11.81081,0
Brazil,69.64803,1
Canada,61.94543,1
China,19.75491,1
Germany,17.90153,1
Spain,37.96859,1
France,17.02028,1
United Kingdom,10.60047,0
India,14.96908,0


## Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

In [19]:
ContinentDict = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [20]:
# only the useful columns
Continent = Merge
Continent['Population'] = (Continent['Energy Supply']/Continent['Energy Supply per Capita']).astype(float)
Continent = Continent.reset_index()
# let's assign continent to each country
Continent['Continent'] = [ContinentDict[x] for x in Continent['Country']]
# and grouping by Continent we get the following data
Continent2 = Continent.set_index('Continent').groupby(level=0)['Population'].agg(size='size', sum='sum', mean='mean', std='std')
Continent

,Country,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,...,2008,2009,2010,2011,2012,2013,2014,2015,Population,Continent
0,Australia,14.0,8831.0,8725.0,90765.0,15606.0,10.28,107.0,5386000000,231,...,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12,2.331602e+07,Australia
1,Brazil,15.0,8668.0,8596.0,60702.0,14396.0,7.00,86.0,12149000000,59,...,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12,2.059153e+08,South America
2,Canada,6.0,17899.0,17620.0,215003.0,40930.0,12.01,149.0,10431000000,296,...,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,3.523986e+07,North America
3,China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,127191000000,93,...,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,1.367645e+09,Asia
4,Germany,7.0,17027.0,16831.0,140566.0,27426.0,8.26,126.0,13261000000,165,...,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,8.036970e+07,Europe
5,Spain,12.0,9428.0,9330.0,123336.0,23964.0,13.08,115.0,4923000000,106,...,1.484530e+12,1.431475e+12,1.431673e+12,1.417355e+12,1.380216e+12,1.357139e+12,1.375605e+12,1.419821e+12,4.644340e+07,Europe
6,France,9.0,13153.0,12973.0,130632.0,28601.0,9.93,114.0,10597000000,166,...,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,6.383735e+07,Europe
7,United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7920000000,124,...,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,6.387097e+07,Europe
8,India,8.0,15005.0,14841.0,128763.0,37209.0,8.58,115.0,33195000000,26,...,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.276731e+09,Asia
9,Iran,13.0,8896.0,8819.0,57470.0,19125.0,6.46,72.0,9172000000,119,...,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN,7.707563e+07,Asia


## Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

In [27]:
# Earlier proccess helps us
Renewable2 = Merge
Renewable2 = Renewable2.reset_index()
Renewable2['Continent'] = [ContinentDict[x] for x in Renewable2['Country']]
Renewable2['% Renewable'] = pd.cut(Renewable2['% Renewable'], 5)
Renewable2 = Renewable2.groupby(['Continent', '% Renewable']).size()
Renewable2

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

## Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

In [30]:
# let's create a function that performa that
def convert_to_string(x):
    num_str = "{:,}".format(x)
    return num_str
# Now let's deal with population column
Populace = Merge[['Population']]
Populace = Populace['Population'].apply(lambda x: convert_to_string(x))
Populace

Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
Germany                 80,369,696.96969697
Spain                    46,443,396.2264151
France                  63,837,349.39759036
United Kingdom         63,870,967.741935484
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
South Korea            49,805,429.864253394
Russian Federation            143,500,000.0
United States          317,615,384.61538464
Name: Population, dtype: object